## 資料清洗函式

In [1]:
import re

slash_annotation_language = ('C', 'C++', 'Go', 'Java', 'JavaScript')
hashtag_annotation_language = ('Bash', 'PowerShell', 'Python', 'Ruby', 'Rust')
def remove_slash_annotation(code):
    # 去除行前空格
    code_without_annotation = re.sub(r'^ {1,}', '', code, flags=re.MULTILINE)  
    # 去除所有縮排
    code_without_annotation = re.sub(r'\t+', '', code_without_annotation)
    # 去除所有單行註解
    code_without_annotation = re.sub(r'\/\/.*', '', code_without_annotation)
    # 去除所有多行註解 (閉合的)
    code_without_annotation = re.sub(r'\/\*[\s\S]*?\*\/', '', code_without_annotation, flags=re.DOTALL)
    # 去除非閉合多行註解
    code_without_annotation = re.sub(r'\/\*[\s\S]*', '', code_without_annotation)
    code_without_annotation = re.sub(r'.*?\*\/', '', code_without_annotation, flags=re.DOTALL)
    # 去除所有無意義換行
    code_without_annotation = re.sub(r'\n+', '\n', code_without_annotation)
    # 去除前後兩端無意義換行
    code_without_annotation = code_without_annotation.strip('\n')
    return code_without_annotation

def remove_hashtag_annotation(code):
    # 去除行前空格
    code_without_annotation = re.sub(r'^ {2,}', '', code, flags=re.MULTILINE)  
    # 去除所有縮排
    code_without_annotation = re.sub(r'\t+', '', code_without_annotation)
    # 去除所有單行註解
    code_without_annotation = re.sub(r'#.*?', '', code_without_annotation, flags=re.MULTILINE)
    # 去除所有無意義換行
    code_without_annotation = re.sub(r'\n+', '\n', code_without_annotation)
    # 去除前後兩端無意義換行
    code_without_annotation = code_without_annotation.strip('\n')
    return code_without_annotation


def remove_annotation(code, language):
    #print(f"原始程式碼:\n{code}", end='')
    if language in slash_annotation_language:
        code_without_annotation = remove_slash_annotation(code)
    elif language in hashtag_annotation_language:
        code_without_annotation = remove_hashtag_annotation(code)
    #print(f"清洗過後的程式碼:\n{code_without_annotation}")
    return code_without_annotation


## 資料提取(無須執行，已產生檔案)

In [2]:
import sqlite3

conn = sqlite3.connect('../snippets/snippets.db') #僅第一次執行須下載檔案
cursor = conn.cursor()
code_type_list = ('Bash', 'C', 'C++', 'Go', 'Java', 'JavaScript', 'PowerShell', 'Python', 'Ruby', 'Rust')

for code_type in code_type_list:
    cursor.execute(f"select snippet from snippets where language = '{code_type}' limit 10000;")
    rows = cursor.fetchall()
    with open(f'../snippets/{code_type}.txt', 'w', encoding='utf-8') as f:
        for row in rows:
            f.write(row[0])
    print(f"{code_type}.txt create successfully.")

Bash.txt create successfully.
C.txt create successfully.
C++.txt create successfully.
Go.txt create successfully.
Java.txt create successfully.
JavaScript.txt create successfully.
PowerShell.txt create successfully.
Python.txt create successfully.
Ruby.txt create successfully.
Rust.txt create successfully.


## 資料清洗

In [3]:
import random

x_train, y_train, x_test, y_test = [], [], [], []
code_type_list = ('Bash', 'C', 'C++', 'Go', 'Java', 'JavaScript', 'PowerShell', 'Python', 'Ruby', 'Rust')

for code_type in code_type_list:
    with open(f"../snippets/{code_type}.txt", 'r', encoding="utf-8") as f:
        lines = f.readlines()
        rows = [''.join(lines[i:i+5]) for i in range(0, len(lines), 5)]
        random.shuffle(rows)
        train_size = int(len(rows) * 0.8)
        train_data = rows[:train_size]
        test_data = rows[train_size:]
        label = code_type
        for data in train_data:
            clean_data = remove_annotation(data, label)
            if clean_data != "":
                x_train.append(clean_data)
                y_train.append(label)

        for data in test_data:
            clean_data = remove_annotation(data, label)
            if clean_data != "":
                x_test.append(clean_data)
                y_test.append(label)


## 資料特徵化 (bage of word)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_train_features_BoW = vectorizer.fit_transform(x_train)
x_test_features_Bow = vectorizer.transform(x_test)

## 資料特徵化 (TF-IDF)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
x_train_features_TF_IDF = vectorizer.fit_transform(x_train)
x_test_features_TF_IDF = vectorizer.transform(x_test)

## 模型訓練 (LogisticRegression)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = LogisticRegression(max_iter=300)
model.fit(x_train_features_TF_IDF, y_train)

train_predictions = model.predict(x_train_features_TF_IDF)
test_predictions = model.predict(x_test_features_TF_IDF)

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions, average='macro', zero_division=1)
train_recall = recall_score(y_train, train_predictions, average='macro', zero_division=1)
train_f1 = f1_score(y_train, train_predictions, average='macro', zero_division=1)
print(f"訓練集準確率: {train_accuracy:.5f}")
print(f"訓練集精確率: {train_precision:.5f}")
print(f"訓練集召回率: {train_recall:.5f}")
print(f"訓練集F1指標: {train_f1:.5f}")
print("="*20)
test_accuracy = accuracy_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions, average='macro', zero_division=1)
test_recall = recall_score(y_test, test_predictions, average='macro', zero_division=1)
test_f1 = f1_score(y_test, test_predictions, average='macro', zero_division=1)
print(f"測試集準確率: {test_accuracy:.5f}")
print(f"測試集精確率: {test_precision:.5f}")
print(f"測試集召回率: {test_recall:.5f}")
print(f"測試集F1指標: {test_f1:.5f}")

# print("預測失敗之結果、原本的正確答案和原始代碼：")
# for pred, true_label, code in zip(test_predictions, y_test, x_test):
#     if (pred != true_label):
#         print(f"預測结果: {pred}\t原本的正確答案: {true_label}\t原始代碼:\n{code}")

訓練集準確率: 0.96174
訓練集精確率: 0.96129
訓練集召回率: 0.96083
訓練集F1指標: 0.96092
測試集準確率: 0.93122
測試集精確率: 0.93120
測試集召回率: 0.92985
測試集F1指標: 0.93016


## 將訓練結果打包儲存

In [7]:
import pandas as pd
from pathlib import Path

df = pd.DataFrame({"預測結果": test_predictions, "原本的正確答案": y_test})
file_path = r'..\result\results_TF_IDF.xlsx'
path = Path(file_path)
if not path.parent.exists():
    path.parent.mkdir(parents=True)
df.to_excel(file_path, index=False)

## 模型訓練(MultinomialNB)

In [8]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()

classifier.fit(x_train_features_BoW, y_train)

train_predictions = classifier.predict(x_train_features_BoW)

test_predictions = classifier.predict(x_test_features_Bow)

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions, average='macro', zero_division=1)
train_recall = recall_score(y_train, train_predictions, average='macro', zero_division=1)
train_f1 = f1_score(y_train, train_predictions, average='macro', zero_division=1)
print(f"訓練集準確率: {train_accuracy:.5f}")
print(f"訓練集精確率: {train_precision:.5f}")
print(f"訓練集召回率: {train_recall:.5f}")
print(f"訓練集F1指標: {train_f1:.5f}")
print("="*20)
test_accuracy = accuracy_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions, average='macro', zero_division=1)
test_recall = recall_score(y_test, test_predictions, average='macro', zero_division=1)
test_f1 = f1_score(y_test, test_predictions, average='macro', zero_division=1)
print(f"測試集準確率: {test_accuracy:.5f}")
print(f"測試集精確率: {test_precision:.5f}")
print(f"測試集召回率: {test_recall:.5f}")
print(f"測試集F1指標: {test_f1:.5f}")

# print("預測失敗之結果、原本的正確答案和原始代碼：")
# for pred, true_label, code in zip(test_predictions, y_test, x_test):
#     if (pred != true_label):
#         print(f"預測结果: {pred}\t原本的正確答案: {true_label}\t原始代碼:\n{code}")

訓練集準確率: 0.94215
訓練集精確率: 0.94344
訓練集召回率: 0.94020
訓練集F1指標: 0.94102
測試集準確率: 0.93065
測試集精確率: 0.93141
測試集召回率: 0.92854
測試集F1指標: 0.92920


## 將訓練結果打包儲存

In [9]:
import pandas as pd
from pathlib import Path

df = pd.DataFrame({"預測結果": test_predictions, "原本的正確答案": y_test})
file_path = r'..\result\results_BoW.xlsx'
path = Path(file_path)
if not path.parent.exists():
    path.parent.mkdir(parents=True)
df.to_excel(file_path, index=False)